In [1]:
from __future__ import print_function
import argparse
import numpy as np
import torch
import math
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time
import torch.optim as optim
from torch.autograd import Variable

In [2]:
batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [3]:
class InceptionA(nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size = 1)
        
        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16,24,kernel_size = 5, padding =2)
        
        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size = 1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size = 3, padding =1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size = 3, padding =1)
        
        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size = 1)
        
    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)
        
        branch_pool = F.avg_pool2d(x, kernel_size = 3, stride = 1, padding = 1)
        branch_pool = self.branch_pool(branch_pool)
        
        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        
        return torch.cat(outputs, 1)

In [4]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size = 5)
        
        self.incept1 = InceptionA(in_channels = 10)
        self.incept2 = InceptionA(in_channels = 20)
        
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)
        
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return F.log_softmax(x)

In [9]:
model = net()
model.to(device)
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

In [10]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [11]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss()
        test_loss += F.nll_loss(output, target, size_average=False).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [ ]:
for epoch in range(1, 10):
    train(epoch)
    test()

<ipython-input-4-ea14773f994e>:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312578
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.304968
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.300013
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.301765
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.286862
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.297586
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.287486
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.303152
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.286756
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.285491
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.267778
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.257797
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.251420
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.240700
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.211678
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.169512
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.111501
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.972783
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.798708
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.464570
Train Epoch: 1 [12800/60000 (

C:\Users\ansgh\anaconda3\envs\pytorch17_p38\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.2371, Accuracy: 9254/10000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.454623
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.233276
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.143055
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.251886
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.219599
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.167889
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.393366
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.328309
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.141692
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.206198
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.134557
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.166322
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.256308
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.206316
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.099062
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.164992
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.395912
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.132481
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.276699
Train Epoch: 2 [12